### Symbolic treatment of the fluid equations

In this notebook we obtain the equations, and all relevant quantities starting from the generating function and in arbitrary dimensions. 
We spacialize for the conformal case, but this is not needed. 

We use a flat, constant metric, but it is not difficult to add an arbitrary metric latter on, even curvature.

$$
T^{ab} = \frac{\partial^2 \chi}{\zeta_a \zeta_b}
$$

$$
A^{abc} = \frac{\partial^2 \chi}{\zeta_a \zeta_{bc}}
$$

In our case the conformally invariant $\chi$ is given by:

$$
\chi(\zeta_a, \zeta_{ab}) = \chi_0 \mu^{-d/2 + 1} + \chi_1 \nu \mu^{-d/2 -1} 
+ \chi_2(\tau_2 - \frac{4(d/2+1)}{\mu} \omicron + \frac{2(d/2+1)(d/2+2)}{\mu^2}\nu^2)\mu^{-d/2 -1},
$$

where,

$$
\mu := \zeta^a \zeta_a \;\;\;\;\;\;\;\;  \nu := \zeta_{ab}\zeta^a \zeta^b \;\;\;\;\;\;\;\; \omicron := \zeta^a \zeta_{ab} \zeta^{bc} \zeta_c, \;\;\;\;\;\;\;\; \tau_2  := \zeta_{ab}\zeta^{ab}
$$



In [1]:
using Symbolics
using LinearAlgebra
using Latexify
using Plots

In [2]:
include("symbolics_ext.jl")

tau2 (generic function with 1 method)

The variables are going to be a vector-tensor $\zeta_A = (\zeta_a, \zeta_{ab})$ with the tensor part being symmetric and (latter) trace free. We are going to use both representations, either a big vector or a pair vector-tensor.
With functions going from one to the other.

We first define some tensors and take a symmetrize them using the upper triangular part as the relevant tensor part, contrary to take the average of the components

In [65]:
D = Int64(4) #dimensions
#D = Int64(2) #dimensions
L = (D+(D*(D+1)÷2))
ζ = Vector{Float64}(undef,L)
ζ_v = Vector{Float64}(undef,D)
ζ_t = Array{Float64}(undef,D,D)
g = make_g(D)

4×4 Matrix{Float64}:
 -1.0  0.0  0.0  0.0
  0.0  1.0  0.0  0.0
  0.0  0.0  1.0  0.0
  0.0  0.0  0.0  1.0

In [38]:
v = [1;2;3;4;5;6]
sm = zeros(3,3)

vector2symmat!(v,sm)
sm

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  4.0  5.0
 3.0  5.0  6.0

In [39]:
symmat2vector(sm)

6-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0

In [40]:
vector_unpack(rand(L))

([0.9070927111057722, 0.7043358484604735], [0.8329732507262227 0.9158566651326762; 0.9158566651326762 0.18222620899792474])

In [41]:
v = [1;2;3;4;5;6;7;8;9;10;11;12;13;14]
vector_unpack(v)

([1, 2, 3, 4], [5.0 6.0 7.0 8.0; 6.0 9.0 10.0 11.0; 7.0 10.0 12.0 13.0; 8.0 11.0 13.0 14.0])

In [42]:
ζ_t = rand(D,D)
println(tr(ζ_t'*g'*g*ζ_t))



ζ_v = rand(D)
#ζ_v = [0.; 0.; 0.; 0.]

upper2symm!(ζ_t)

Tr = tr(g*ζ_t)
ζ_t = ζ_t - Tr*g/D

latexify(ζ_t)

tr(g*ζ_t)

0.9178864050156783


0.0

Likewise we define functions which take either a long vector or the pair vector-tensor

In [66]:
function Φ_vt(ζ_v, ζ_s, p)
    D = length(ζ_v)
    g = make_g(D)
    #println(g)
    χ₀ = p[1]
    χ₁ = p[2]
    χ₂ = p[3]
    #ζ_s = my_symmetrization!(Symbolics.scalarize(ζ_t))
    μ = Symbolics.scalarize(ζ_v)'*g*Symbolics.scalarize(ζ_v)
    ν = (g*Symbolics.scalarize(ζ_v))'*Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    l = Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    𝚶 = l'* g * l
    τ₂= tr(Symbolics.scalarize(ζ_s)*g'*Symbolics.scalarize(ζ_s)*g)
    D1 = D//2+1
    D2 = D//2+2
    return χ₀*μ^(1-D//2) + χ₁*ν*μ^(-D1) + χ₂*(τ₂ - 4*D1*𝚶*μ^(-1) + 2*D1*D2*ν^2*μ^(-2))*μ^(-D1) 
    # the function seems to be correct: tr(g*T)=0 and also A is fully symmetric after symmetrization 
    # on first two indices
    #return 𝚶
end

function Φ_v(ζ,p)
    ζ_v, ζ_t = vector_unpack(ζ)
    #ζ_t = vector2symmat(ζ[D+1:end])
    #return Φ_vt(ζ[1:D], ζ_t, p)
    return Φ_vt(ζ_v, ζ_t, p)
end

ζ_v,ζ_t = vector_unpack(v)
Φ_vt(ζ_v,ζ_t, [1.,1.,1.])#, ζ_t, [1.;1;1])
Φ_v(v, [1.,1.,1.])
#ζ_v
#ζ_t

0.6214398204617124

In [67]:
@variables zv[1:D], zs[1:D,1:D], pv[1:3]

Φ_vt(zv, zs, pv);

In [68]:
zvt = [zv[i] for i ∈ 1:D]
zst = vec([zs[i,j] for i ∈ 1:D, j ∈ 1:D])
JΦ_v = Symbolics.gradient(Φ_vt(zv, zs, pv), zvt);
JΦ_s = (reshape(Symbolics.gradient(Φ_vt(zv, zs, pv), zst), (D,D)) + reshape(Symbolics.gradient(Φ_vt(zv, zs, pv), zst), (D,D))')/2 ;

In [69]:
JΦ_v_exp = Symbolics.build_function(JΦ_v,zv, zs, pv);
JΦ_ve = eval(JΦ_v_exp[1]);
JΦ_ve(ζ_v,ζ_t, pv)

4-element Vector{Num}:
  0.002551020408163265pv[1] + 0.0028406132861307776pv[2] + 0.010664103339169867pv[3]
 -0.00510204081632653pv[1] - 0.008778894210745519pv[2] - 0.14496530729117962pv[3]
 -0.007653061224489796pv[1] - 0.015719361724281546pv[2] - 0.31656681766313355pv[3]
 -0.01020408163265306pv[1] - 0.023297584339858384pv[2] - 0.5131969619376278pv[3]

In [70]:
JΦ_s_exp = Symbolics.build_function(JΦ_s, zv, zs, pv);
JΦ_se = eval(JΦ_s_exp[1]);
JΦ_se(ζ_v, ζ_t, pv);

In [71]:
T_s = Symbolics.jacobian(JΦ_ve(zv, zs, pv), zvt);

In [72]:
T_exp = Symbolics.build_function(T_s, zv, zs, pv);
T = eval(T_exp[1]);

In [73]:
tr(g*T(ζ_v, ζ_t, [1.0;1.0;1.0]))

0.14514041768310848

In [74]:
tr(g*ζ_t)

30.0

In [18]:
A_s = Symbolics.jacobian(JΦ_ve(zv, zs, pv), zst);

In [19]:
A_exp = Symbolics.build_function(A_s, zv, zs, pv);
A = eval(A_exp[1]);

In [20]:
A_val = A(ζ_v, ζ_t, [1.0;1.0;1.0]);

In [21]:
AA = reshape(A_val', (D,D,D)); # last index is the vectorial one

In [22]:
AS = symm_A(AA)

3×3×3 Array{Float64, 3}:
[:, :, 1] =
  0.126501  -0.168387   -0.282929
 -0.168387   0.0823454   0.138275
 -0.282929   0.138275    0.233306

[:, :, 2] =
 -0.0367178   0.0282049   0.138275
  0.0282049   0.0641222  -0.109116
  0.138275   -0.109116   -0.549306

[:, :, 3] =
 -0.0620375   0.138275   0.179166
  0.138275   -0.330008  -0.417636
  0.179166   -0.417636  -0.474892

In [23]:
tr_A(AS,g)

3-element Vector{Float64}:
  0.18915070668112088
 -0.44846550857625883
 -0.7428623580784051

In [24]:
tr_A(STF_A(AS,g),g)

3-element Vector{Float64}:
 2.7755575615628914e-17
 0.0
 5.551115123125783e-17

In [25]:
A_stf = STF_A(AS,g)

3×3×3 Array{Float64, 3}:
[:, :, 1] =
  0.189551  -0.168387   -0.282929
 -0.168387   0.0192951   0.138275
 -0.282929   0.138275    0.170256

[:, :, 2] =
 -0.186206    0.0282049   0.138275
  0.0282049   0.213611   -0.109116
  0.138275   -0.109116   -0.399817

[:, :, 3] =
 -0.309658   0.138275    0.179166
  0.138275  -0.0823871  -0.417636
  0.179166  -0.417636   -0.227271

In [26]:
full_symmetry_check(A_stf)

0.0

In [27]:
A_s_dual = Symbolics.jacobian(JΦ_se(zv, zs, pv), zvt);

LoadError: MethodError: no method matching jacobian(::Matrix{Num}, ::Vector{Num})
[0mClosest candidates are:
[0m  jacobian([91m::AbstractVector{T} where T[39m, ::AbstractVector{T} where T; simplify) at /Users/reula/.julia/packages/Symbolics/ohRLi/src/diff.jl:368

We define now the conserved quantities and then the field-coordinate transformation.

In [28]:
function C(ζ,p,g,k::Int64) 
    D = length(Symbolics.scalarize(g)[:,1])
    ζ_v, ζ_t = Symbolics.scalarize(vector_unpack(ζ))
        AA = reshape(Symbolics.scalarize(A(ζ_v, ζ_t, p))', (D,D,D))[:,:,k]
        upper2symm!(AA)
        Tr = tr(AA)
        AA = AA - g*Tr/D
        return [T(ζ_v, ζ_t, p)[:,k]; symmat2vector(AA)]
end

C (generic function with 1 method)

In [29]:
@variables zz[1:D+D*(D+1)÷2] gv[D,D]
zzt = [zz[i] for i ∈ 1:(D+D*(D+1)÷2)]
JC1_s = Symbolics.jacobian(C(zz,pv,gv,1),zzt)

LoadError: ArgumentError: Cannot convert Sym to Float64 since Sym is symbolic and Float64 is concrete. Use `substitute` to replace the symbolic unwraps.

In [30]:
C1 = C(rand(14),[1;1;2],g,1)

9-element Vector{Float64}:
  73.13559125207831
  -8.593985278915984
  22.81411857623393
 -35.48386671842226
  11.79020688337824
  17.1641644623997
   0.6604518816882763
 -23.211754288961643
   3.918533476394268

In [31]:
T0, A0 = vector_unpack(C1)

([73.13559125207831, -8.593985278915984, 22.81411857623393], [-35.48386671842226 11.79020688337824 17.1641644623997; 11.79020688337824 0.6604518816882763 -23.211754288961643; 17.1641644623997 -23.211754288961643 3.918533476394268])

In [32]:
full_symmetry_check(A0)

0.0

In [33]:
function C2A!(ζ,c,p)
    χ, tol, iter_max, g, M, C, J = p
    while norm(C(ζ,χ,g,1) - c) > tol && iter < iter_max
            ζ = ζ - J(ζ,p,g) \ (C(ζ,χ,g,1) - c)
            iter = iter + 1
            if iter == iter_max 
            println("iter_max reached j = $j")
            end
    end
end

C2A! (generic function with 1 method)

In [34]:
p_N = [1.,1.,1.], 10^(-6), g, 1, C1, JC1

LoadError: UndefVarError: JC1 not defined

In [35]:
function c_to_f!(flu, con, p)
    χ, tol, iter_max, N, M = p
    for j ∈ 1:M
        flu[1,j] = -flu[1,j]
        iter = 1
        while F(flu[:,j],con[:,j], χ)'*F(flu[:,j],con[:,j], χ) > tol && iter < iter_max
            flu[:,j] = NR_step!(F, Jac, flu[:,j], con[:,j], χ)
            iter = iter + 1
            if iter == iter_max 
            println("iter_max reached j = $j")
            end
        end
    #println(iter)
        flu[1,j] = -flu[1,j]
    end
    return flu[:,:]
end


c_to_f! (generic function with 1 method)

In [54]:
include("symbolics_ext.jl")

Φ_new(rand(L),([1.0;1.0;1.0],make_g(D)))

11939.973833698416

In [55]:
@variables vs[1:L] ps[1:3] gs[D,D]
vst = [vs[i] for i ∈ 1:L]
pv = (ps, gs)
∇ζ = Symbolics.gradient(Φ_new(vs,pv),vst);

In [56]:
∇ζ_exp = Symbolics.build_function(∇ζ,vs,pv);
∇ζ_f = eval(∇ζ_exp[1]);

In [59]:
g = make_g(D)
pp = ([1.0;1.0;1.0],g)
∇ζ_f(rand(L),([1.0;1.0;1.0],make_g(D)))

MethodError: MethodError: no method matching create_array(::Type{Tuple{Vector{Float64}, Matrix{Float64}}}, ::Nothing, ::Val{1}, ::Val{(5,)}, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64)
Closest candidates are:
  create_array(!Matched::Type{<:LabelledArrays.SLArray}, ::Any, ::Val, ::Val{dims}, ::Any...) where dims at ~/.julia/packages/SymbolicUtils/fgHzN/src/code.jl:498
  create_array(!Matched::Type{<:LabelledArrays.LArray}, ::Any, ::Val, ::Val{dims}, ::Any...) where dims at ~/.julia/packages/SymbolicUtils/fgHzN/src/code.jl:507
  create_array(!Matched::Type{<:PermutedDimsArray{T, N, perm, iperm, P}}, ::Any, ::Val, ::Val, ::Any...) where {T, N, perm, iperm, P} at ~/.julia/packages/SymbolicUtils/fgHzN/src/code.jl:466
  ...

In [11]:
Symbolics.jacobian(∇ζ(vs,pv),vst);

MethodError: MethodError: objects of type Vector{Num} are not callable
Use square brackets [] for indexing an Array.

In [37]:
ζ_v, ζ_t = vector_unpack(vs)
#ζ_v' * gs * ζ_v

#typeof(ζ_v)

LoadError: ArgumentError: Cannot convert Sym to Float64 since Sym is symbolic and Float64 is concrete. Use `substitute` to replace the symbolic unwraps.

In [38]:
Symbolics.scalarize(vs)[1:D]

3-element Vector{Num}:
 vs[1]
 vs[2]
 vs[3]

In [39]:
vector2symmat(Symbolics.scalarize(vs)[D+1:end])[:,:]

LoadError: ArgumentError: Cannot convert Sym to Float64 since Sym is symbolic and Float64 is concrete. Use `substitute` to replace the symbolic unwraps.

In [40]:
include("symbolics_ext.jl")
mu(rand(14))
nu(rand(14))
omicron(rand(14))
tau2(rand(14))

-0.34011986318045284